In [1]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
#from torch.optim.lr_scheduler import _LRScheduler

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torchvision import models
from torchsummary import summary
import torch.nn.functional as F


/home/MMI24limbyungwan/.conda/envs/byungwan_resn/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# GPU 사용이 가능할 경우, GPU를 사용할 수 있게 함.'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

print(os.environ.get('CUDA_VISIBLE_DEVICES'))

cuda
2


In [3]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 클래스별 IoU를 계산하기 위한 함수
def calculate_iou_per_class(y_true, y_pred, class_id):
    intersection = np.sum((y_true == class_id) & (y_pred == class_id))
    union = np.sum((y_true == class_id) | (y_pred == class_id))
    iou = intersection / union if union > 0 else 0
    return iou

In [4]:
import numpy as np
import cv2
import torch

def apply_fisheye_distortion(images, masks, label):
    # 이미지 크기 가져오기
    batch, channel, height, width = images.shape

    # 카메라 매트릭스 생성
    focal_length = width / 4
    center_x = width / 2
    center_y = height / 2
    camera_matrix = np.array([[focal_length, 0, center_x],
                              [0, focal_length, center_y],
                              [0, 0, 1]], dtype=np.float32)

    # 왜곡 계수 생성
    # dist_num = 0
    # if label == 1:
    #     dist_num = random.randint(1,3)
    # elif label == 2.5:
    #     dist_num = 2.5
    dist_num = label
    dist_coeffs = np.array([0, 0.1 * dist_num, 0, 0], dtype=np.float32)

    # 왜곡 보정
    undistorted_images = []
    undistorted_masks = []

    for i in range(batch):
        image = images[i].permute(1, 2, 0).cpu().numpy()  # 텐서를 NumPy 배열로 변환
        mask = masks[i].cpu().numpy()
        undistorted_image = cv2.undistort(image, camera_matrix, dist_coeffs)
        undistorted_mask = cv2.undistort(mask, camera_matrix, dist_coeffs)
        undistorted_mask = np.round(undistorted_mask).astype(np.uint8)
        undistorted_mask[undistorted_mask > 12] = 12

        # 다시 텐서로 변환
        undistorted_image = torch.from_numpy(undistorted_image).permute(2, 0, 1).float().to(device)
        undistorted_mask = torch.from_numpy(undistorted_mask).long().to(device)

        undistorted_images.append(undistorted_image)
        undistorted_masks.append(undistorted_mask)

    undistorted_images = torch.stack(undistorted_images, dim=0)
    undistorted_masks = torch.stack(undistorted_masks, dim=0)

    return undistorted_images, undistorted_masks


In [5]:
# import torch

# # 이미지 데이터 (batch, channel, height, width)
# image_data = torch.randn(4, 3, 256, 256)

# # 이미지 차원 변경 (batch, height, width, channel)
# image_data_permuted = image_data.permute(0, 2, 3, 1)

# # 이미지 크기 확인
# print(image_data_permuted.shape)

# # 4개의 이미지로 나누기
# images = image_data_permuted.split(1, dim=0)
# # 또는 images = torch.split(image_data_permuted, 1, dim=0)
# undistorted_images = []
# # 4개 이미지의 크기 확인
# for i, image in enumerate(images):
#     print(f"Image {i + 1} shape: {image.shape}")
#     undistorted_images.append(images[i].squeeze())

# undistorted_images2 = torch.stack(undistorted_images, dim=0)
# undistorted_images3 = undistorted_images2.permute(0,3,1,2)

In [6]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #directory_path = "/mnt/nas27/Dataset/Samsung_DM"
        directory_path = './data/224'
        img_path = self.data.iloc[idx, 1]
        img_path = os.path.join(directory_path, img_path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        mask_path = self.data.iloc[idx, 2]
        mask_path = os.path.join(directory_path, mask_path)
        mask = cv2.imread(mask_path)
        #mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask = np.round(mask).astype(np.uint8)
        mask[mask > 12] = 12 #배경을 픽셀값 12로 간주
        mask += 1
        mask[mask == 13] = 0

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

# class CustomDataset_target(Dataset):
#     def __init__(self, csv_file, transform=None, infer=False):
#         self.data = pd.read_csv(csv_file)
#         self.transform = transform
#         self.infer = infer

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         directory_path = "/mnt/nas27/Dataset/Samsung_DM"
#         img_path = self.data.iloc[idx, 1]
#         img_path = os.path.join(directory_path, img_path[2:])
#         image = cv2.imread(img_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
#         if self.infer:
#             if self.transform:
#                 image = self.transform(image=image)['image']
#             return image


#         if self.transform:
#             augmented = self.transform(image=image)
#             image = augmented['image']
            

#         return image
     

transform = A.Compose(
    [   
        #A.Resize(224, 224),
        #A.Resize(128, 128),
        A.Normalize(),
        
        # 변형
        # A.VerticalFlip(p=0.5),
        # A.RandomRotate90(p=0.5),
        # A.HueSaturationValue(p=0.2),
        
        ToTensorV2()
    ]
)

In [7]:
# #Unet의 기본이 되는 conv블럭
# class ConvBlock(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(ConvBlock, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
#         self.bn1 = nn.BatchNorm2d(out_channels)
#         self.relu1 = nn.ReLU()
        
#         self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)  # 여기서 in_channels는 out_channels와 동일해야 합니다.
#         self.bn2 = nn.BatchNorm2d(out_channels)
#         self.relu2 = nn.ReLU()

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu1(x)
        
#         x = self.conv2(x)
#         x = self.bn2(x)
#         x = self.relu2(x)
#         return x

# #인코더 블럭
# class EncoderBlock(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(EncoderBlock,self).__init__()
#         self.convblock1 = ConvBlock(in_channels, out_channels)  # 첫 번째 ConvBlock의 in_channels는 입력 이미지의 채널 수와 일치해야 합니다.
#         #self.convblock2 = ConvBlock(out_channels, out_channels)  # 두 번째 ConvBlock의 in_channels는 out_channels와 일치해야 합니다.
#         self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

#     def forward(self,x):
#         x = self.convblock1(x)
#         #x = self.convblock2(x)
#         p = self.maxpool(x)
#         return x , p
# #디코더 블럭
# #디코더는 업샘플링 이후 스킵연결과 붙어서 convblock을 통과해야함
# #skip보다 작은 x x먼저 업샘플링 32 -> 64 , skip과 결합 6464 
# class DecoderBlock(nn.Module):
#     def __init__(self, channels):
#         super(DecoderBlock,self).__init__()
#         self.upsample = nn.ConvTranspose2d(channels*2, channels, kernel_size=4, stride=2, padding=1)#x 업샘플링
#         self.convblock1 = ConvBlock(channels*2, channels)#차원감소
#         #self.convblock2 = ConvBlock(channels, channels)
#     def forward(self,x,skip):
#         x = self.upsample(x)
#         x = torch.cat([x, skip], dim=1)
#         x = self.convblock1(x)
#         #x = self.convblock2(x)
#         return x

# ###########################################
# class GradReverse(torch.autograd.Function):
#     @staticmethod
#     def forward(self, x):
#         return x.view_as(x)
#     @staticmethod
#     def backward(self, grad_output): # 역전파 시에 gradient에 음수를 취함
#         return (grad_output * -1)

# class domain_classifier(nn.Module):
#     def __init__(self):
#         super(domain_classifier, self).__init__()
#         self.fc1 = nn.Linear(224*224*64, 10)
#         self.fc2 = nn.Linear(10, 4) # source = 0, target = 1 회귀 가정

#     def forward(self, x):
#         x = x.view(-1, 224*224*64)
#         x = GradReverse.apply(x) # gradient reverse
#         x = F.leaky_relu(self.fc1(x))
#         x = self.fc2(x)
        
#         return x

# ###########################################


# #Unet구조 middle의 xm값의 움직임에 주의
# class Unet(nn.Module):
#     def __init__(self,n_classes):
#         super(Unet,self).__init__()
#         self.encoder1 = EncoderBlock(3,64)
#         self.encoder2 = EncoderBlock(64,128)
#         self.encoder3 = EncoderBlock(128,256)
#         self.encoder4 = EncoderBlock(256,512)
        
#         self.middleconv = ConvBlock(512,1024)
        
        
#         self.decoder4 = DecoderBlock(512)
#         self.decoder3 = DecoderBlock(256)
#         self.decoder2 = DecoderBlock(128)
#         self.decoder1 = DecoderBlock(64)
#         self.segmap = nn.Conv2d(64,n_classes, kernel_size=1)
        
#         self.domain_classifier = domain_classifier()
                                        

#     def forward(self,x):
#         x1,p = self.encoder1(x)#3->64   #P:256,256 x1 :512,512
#         x2,p = self.encoder2(p)#64->128 #P:128,128 x2:256,256
#         x3,p = self.encoder3(p)#128->256#p:64,64 x3:128,128
#         x4,p = self.encoder4(p)#256->512#p:32,32 x4:64,64
        
#         xm = self.middleconv(p)#512->1024#32,32
        
#         x = self.decoder4(xm,x4)#뉴런:1024->512->512 #출력tensor:64,64
#         x = self.decoder3(x,x3)#뉴런:512->256->256 #출력tensor:128,128
#         x = self.decoder2(x,x2)#뉴런:256->128->128 #출력tensor:256,256
#         x = self.decoder1(x,x1)#뉴런:128->64->64 #출력tensor:512,512

#         x_c = self.segmap(x)
#         x_d = self.domain_classifier(x)
#         # print("x_c", x_c.shape)
#         # print("x_d", x_d.shape)
#         return x_c, x_d

## Resnet50

In [8]:
# class GradReverse(torch.autograd.Function):
#     @staticmethod
#     def forward(self, x):
#         return x.view_as(x)
#     @staticmethod
#     def backward(self, grad_output): # 역전파 시에 gradient에 음수를 취함
#         return grad_output * (-1)

# class domain_classifier(nn.Module):
#     def __init__(self):
#         super(domain_classifier, self).__init__()
#         self.conv1 = nn.Conv2d(128, 1, kernel_size=1)
#         self.fc1 = nn.Linear(224*224*1, 10)
#         self.fc2 = nn.Linear(10, 4) # source = 0, target = 1 회귀 가정

#     def forward(self, x):
#         x = self.conv1(x)
#         x = x.view(-1, 224*224*1)
#         #print(x.shape)
#         x = GradReverse.apply(x) # gradient reverse
#         x = F.leaky_relu(self.fc1(x))
#         x = self.fc2(x)
#         #print(x.shape)
#         #return torch.sigmoid(x)
#         return x

# #Unet의 기본이 되는 conv블럭
# class ConvBlock(nn.Module):
#     def __init__(self, in_channels, out_channels,kernel_size = 3):
#         super(ConvBlock, self).__init__()
#         self.kernel_size = kernel_size
#         self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=1)
#         self.bn1 = nn.BatchNorm2d(out_channels)
#         self.relu1 = nn.ReLU()
        
#         self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=1)  # 여기서 in_channels는 out_channels와 동일해야 합니다.
#         self.bn2 = nn.BatchNorm2d(out_channels)
#         self.relu2 = nn.ReLU()

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu1(x)
        
#         x = self.conv2(x)
#         x = self.bn2(x)
#         x = self.relu2(x)
#         return x
# class IdentityBlock(nn.Module):
#     def __init__(self, in_channels, mid_channels, out_channels, stride=1):
#         super(IdentityBlock, self).__init__()
        
#         # 1x1 convolution
#         self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=1, stride=stride, bias=False)
#         self.bn1 = nn.BatchNorm2d(mid_channels)
#         self.relu1 = nn.ReLU()

#         # 3x3 convolution
#         self.conv2 = nn.Conv2d(mid_channels, mid_channels, kernel_size=3, padding=1, bias=False)
#         self.bn2 = nn.BatchNorm2d(mid_channels)
#         self.relu2 = nn.ReLU()

#         # 1x1 convolution
#         self.conv3 = nn.Conv2d(mid_channels, out_channels, kernel_size=1, bias=False)
#         self.bn3 = nn.BatchNorm2d(out_channels)
#         self.relu3 = nn.ReLU()
        
#     def forward(self, x):
#         out = self.conv1(x)
#         out = self.bn1(out)
#         out = self.relu1(out)
        
#         out = self.conv2(out)
#         out = self.bn2(out)
#         out = self.relu2(out)
         
#         out = self.conv3(out)
#         out = self.bn3(out)
#         out = self.relu3(out)
        
#         return out
# class HeadBlock(IdentityBlock):
#     def __init__(self, in_channels, mid_channels, out_channels, stride=1):
#         super(HeadBlock, self).__init__(in_channels, mid_channels, out_channels, stride)
        
#         self.shortcut = nn.Sequential(
#             nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
#             nn.BatchNorm2d(out_channels)
#         )

#     def forward(self, x):
#         identity = x
#         out = super().forward(x)
        
#         if identity.size() != out.size():
#             identity = F.interpolate(identity, size=out.size()[2:])
#         identity = self.shortcut(identity)
        
#         out += identity
#         out = self.relu3(out)
        
#         return out
# #인코더 블럭
# class Conv2(nn.Module):
#     def __init__(self,in_channels, mid_channels, out_channels):
#         super(Conv2,self).__init__() 
#         self.headblock = HeadBlock(in_channels,mid_channels,out_channels)
#         self.identityblock1 = IdentityBlock(out_channels,mid_channels,out_channels)
#         self.identityblock2 = IdentityBlock(out_channels,mid_channels,out_channels)
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
#     def forward(self,x):
#         x = self.headblock(x)
#         x = self.identityblock1(x)
#         x = self.identityblock2(x)
#         p = self.maxpool(x)
#         return x , p
# class Conv3(nn.Module):
#     def __init__(self,in_channels, mid_channels, out_channels):
#         super(Conv3,self).__init__() 
#         self.headblock = HeadBlock(in_channels,mid_channels,out_channels)
#         self.identityblock1 = IdentityBlock(out_channels,mid_channels,out_channels)
#         self.identityblock2 = IdentityBlock(out_channels,mid_channels,out_channels)
#         self.identityblock3 = IdentityBlock(out_channels,mid_channels,out_channels)
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
#     def forward(self,x):
#         x = self.headblock(x)
#         x = self.identityblock1(x)
#         x = self.identityblock2(x)
#         x = self.identityblock3(x)
#         p = self.maxpool(x)
#         return x , p
# class Conv4(nn.Module):
#     def __init__(self,in_channels, mid_channels, out_channels):
#         super(Conv4,self).__init__() 
#         self.headblock = HeadBlock(in_channels,mid_channels,out_channels)
#         self.identityblock1 = IdentityBlock(out_channels,mid_channels,out_channels)
#         self.identityblock2 = IdentityBlock(out_channels,mid_channels,out_channels)
#         self.identityblock3 = IdentityBlock(out_channels,mid_channels,out_channels)
#         self.identityblock4 = IdentityBlock(out_channels,mid_channels,out_channels)
#         self.identityblock5 = IdentityBlock(out_channels,mid_channels,out_channels)
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
#     def forward(self,x):
#         x = self.headblock(x)
#         x = self.identityblock1(x)
#         x = self.identityblock2(x)
#         x = self.identityblock3(x)
#         x = self.identityblock4(x)
#         x = self.identityblock5(x)
#         p = self.maxpool(x)
#         return x , p
# class Conv5(nn.Module):
#     def __init__(self,in_channels, mid_channels, out_channels):
#         super(Conv5,self).__init__() 
#         self.headblock = HeadBlock(in_channels,mid_channels,out_channels)
#         self.identityblock1 = IdentityBlock(out_channels,mid_channels,out_channels)
#         self.identityblock2 = IdentityBlock(out_channels,mid_channels,out_channels)
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
#     def forward(self,x):
#         x = self.headblock(x)
#         x = self.identityblock1(x)
#         x = self.identityblock2(x)
#         p = self.maxpool(x)
#         return x , p
# #디코더 블럭
# class DecoderBlock(nn.Module):
#     def __init__(self, channels):
#         super(DecoderBlock, self).__init__()
#         self.upsample = nn.ConvTranspose2d(channels*2, channels, kernel_size=4, stride=2, padding=1, output_padding=1) # output_padding 추가
#         self.convblock1 = ConvBlock(channels*2, channels)

#     def forward(self, x, skip):
#         x = self.upsample(x)
#         if x.size(2) != skip.size(2) or x.size(3) != skip.size(3):
#             x = F.interpolate(x, size=(skip.size(2), skip.size(3)))
#         x = torch.cat([x, skip], dim=1)
#         x = self.convblock1(x)
#         return x

# #Unet구조 middle의 xm값의 움직임에 주의
# class Resnet50_Unet(nn.Module):
#     def __init__(self,n_classes):
#         super(Resnet50_Unet,self).__init__()
#         self.fconv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2)
#         self.fbn1 = nn.BatchNorm2d(64)
#         self.frelu1 = nn.ReLU()
#         self.fconv2 = nn.Conv2d(64, 128, kernel_size=1, stride=1)
#         self.fbn2 = nn.BatchNorm2d(128)
#         self.frelu2 = nn.ReLU()
#         self.fmaxpooling = nn.MaxPool2d(kernel_size=3,stride=2)
        
#         self.conv2 = Conv2(128,64,256)
#         self.conv3 = Conv3(256,128,512)
#         self.conv4 = Conv4(512,256,1024)
#         self.conv5 = Conv5(1024,512,2048)
        
#         self.middleconv = ConvBlock(2048,4096)
#         self.dropout = nn.Dropout2d(0.4) #
           
#         self.decoder5 = DecoderBlock(2048)
#         self.decoder4 = DecoderBlock(1024)
#         self.decoder3 = DecoderBlock(512)
#         self.decoder2 = DecoderBlock(256)
#         self.decoder1 = DecoderBlock(128)
        
#         self.segmap = nn.Conv2d(128,n_classes, kernel_size=1)
#         self.domain_classifier = domain_classifier()
        
#     def forward(self,x):
#         x = self.fconv1(x)#3->64
#         x = self.fbn1(x)
#         x = self.frelu1(x)
#         x = self.fconv2(x)
#         x = self.fbn2(x)
#         x1 = self.frelu2(x)
#         p = self.fmaxpooling(x)#첫 conv: x0([8, 64, 109, 109]) p([8, 64, 54, 54])
#         x2,p = self.conv2(p)#conv2:  x1:([8, 256, 54, 54]) p([8, 256, 26, 26])
#         x3,p = self.conv3(p)#conv3:  x2([8, 512, 26, 26]) p([8, 512, 12, 12])
#         x4,p = self.conv4(p)#conv4:  x3([8, 1024, 12, 12]) p([8, 1024, 5, 5])
#         x5,p = self.conv5(p)#conv5:  x4([8, 2048, 5, 5]) p([8, 2048, 2, 2])
        
#         xm = self.middleconv(p)#xm([8, 4096, 2, 2])
#         xm = self.dropout(xm)
        
#         x = self.decoder5(xm,x5)#뉴런:2048*2->2048 1
#         x = self.decoder4(x,x4)#뉴런:1024*2->1024 
#         x = self.decoder3(x,x3) #14
#         x = self.decoder2(x,x2)#28
#         x = self.decoder1(x,x1)#55
        
#         x = F.interpolate(x, size=(224, 224))
#         x_c = self.segmap(x)
#         x_d = self.domain_classifier(x)
#         #print(x.shape)
#         return x_c,x_d

## Resnet18

In [9]:
class GradReverse(torch.autograd.Function):
    @staticmethod
    def forward(self, x):
        return x.view_as(x)
    @staticmethod
    def backward(self, grad_output): # 역전파 시에 gradient에 음수를 취함
        return grad_output * (-1)

class domain_classifier(nn.Module):
    def __init__(self):
        super(domain_classifier, self).__init__()
        self.fc1 = nn.Linear(224*224*64, 10)
        self.fc2 = nn.Linear(10, 4) # source = 0, target = 1 회귀 가정

    def forward(self, x):
        x = x.view(-1, 224*224*64)
        x = GradReverse.apply(x) # gradient reverse
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        
        #return torch.sigmoid(x)
        return x

class IdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(IdentityBlock, self).__init__()
        
        # 3x3 convolution
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu1 = nn.ReLU()
        
        # 3x3 convolution
        self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU()
        
        # Skip connection
        self.skip = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        # Adding the skip connection
        out += self.skip(identity)
        out = self.relu2(out)
        
        return out


#인코더 블럭
class Conv2(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(Conv2,self).__init__() 
        self.identityblock1 = IdentityBlock(in_channels,in_channels)
        self.identityblock2 = IdentityBlock(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
    def forward(self,x):
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        p = self.maxpool(x)
        
        return x , p
class Conv3(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(Conv3,self).__init__()         
        self.identityblock1 = IdentityBlock(in_channels,in_channels)
        self.identityblock2 = IdentityBlock(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
    def forward(self,x):
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        p = self.maxpool(x)
        
        return x , p
class Conv4(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(Conv4,self).__init__()         
        self.identityblock1 = IdentityBlock(in_channels,in_channels)
        self.identityblock2 = IdentityBlock(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
    def forward(self,x):
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        p = self.maxpool(x)
        
        return x , p
class Conv5(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(Conv5,self).__init__() 
        self.identityblock1 = IdentityBlock(in_channels,in_channels)
        self.identityblock2 = IdentityBlock(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
        
    def forward(self,x):
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        p = self.maxpool(x)
        
        return x , p
#디코더 블럭
class DecoderBlock(nn.Module):
    def __init__(self, channels):
        super(DecoderBlock, self).__init__()
        self.upsample = nn.ConvTranspose2d(channels*2, channels, kernel_size=4, stride=2, padding=1) # output_padding 추가
        self.convblock1 = IdentityBlock(channels*2, channels)

    def forward(self, x, skip):
        x = self.upsample(x)
        if x.size(2) != skip.size(2) or x.size(3) != skip.size(3):
            x = F.interpolate(x, size=(skip.size(2), skip.size(3)))
        x = torch.cat([x, skip], dim=1)
        x = self.convblock1(x)
        #print("x",x.shape,"skip: ",skip.shape)
        return x

#Unet구조 middle의 xm값의 움직임에 주의
class Resnet18_Unet(nn.Module):
    def __init__(self,n_classes):
        super(Resnet18_Unet,self).__init__()
        self.fconv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.fbn1 = nn.BatchNorm2d(64)
        self.frelu1 = nn.ReLU()
        self.fmaxpooling = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        
        self.conv2 = Conv2(64,128)
        self.conv3 = Conv3(128,256)
        self.conv4 = Conv4(256,512)
        self.conv5 = Conv5(512,1024)
        
        self.middleconv = IdentityBlock(1024,2048)
        self.dropout = nn.Dropout2d(0.1) #
           
        self.decoder5 = DecoderBlock(1024)
        self.decoder4 = DecoderBlock(512)
        self.decoder3 = DecoderBlock(256)
        self.decoder2 = DecoderBlock(128)
        self.decoder1 = DecoderBlock(64)
        self.transpose = nn.ConvTranspose2d(64, 64, kernel_size=4, stride=2, padding=1) # output_padding 추가
        
        self.segmap = nn.Conv2d(64,n_classes, kernel_size=1)
        self.domain_classifier = domain_classifier()
        
    def forward(self,x):
        x = self.fconv1(x)#3->64
        x0 = self.fbn1(x)
        x1 = self.frelu1(x)
        p = self.fmaxpooling(x1)#첫 conv: x0([8, 64, 109, 109]) p([8, 64, 54, 54])
        #print("conv1: ",x1.shape, "maxpooling: ",p.shape)
        x2,p = self.conv2(p)
        #print("conv2: ",x2.shape, "maxpooling: ",p.shape)
        x3,p = self.conv3(p)
        #print("conv3: ",x3.shape, "maxpooling: ",p.shape)
        x4,p = self.conv4(p)
        #print("conv4: ",x4.shape, "maxpooling: ",p.shape)
        x5,p = self.conv5(p)
        #print("conv5: ",x5.shape, "maxpooling: ",p.shape)
        
        xm = self.middleconv(p)#xm([8, 4096, 2, 2])
        #print("xm: ",xm.shape, "maxpooling: ",p.shape)
        xm = self.dropout(xm)
        
        x = self.decoder5(xm,x5)#뉴런:2048*2->2048 1
        x = self.decoder4(x,x4)#뉴런:1024*2->1024 
        x = self.decoder3(x,x3) #14
        x = self.decoder2(x,x2)#28
        x = self.decoder1(x,x1)#55
        x = self.transpose(x)
        
        #print(x.shape)
        #x = F.interpolate(x, size=(224, 224))
        x_c = self.segmap(x)
        x_d = self.domain_classifier(x)
        
        return x_c,x_d

In [10]:
# import pickle

# # 저장된 class_weights를 불러옵니다.
# class_weights_path = 'CLASS_WEIGHTS.pkl'

# with open(class_weights_path, 'rb') as file:
#     CLASS_WEIGHTS = pickle.load(file)

# print(CLASS_WEIGHTS)

## Loss Function

In [11]:
# loss function과 optimizer 정의

class DANN_Loss(nn.Module):
    def __init__(self):
        super(DANN_Loss, self).__init__()

        #self.CE = nn.CrossEntropyLoss(weight=CLASS_WEIGHTS) # 0~9 class 분류용
        self.CE = nn.CrossEntropyLoss()
        #self.BCE = nn.BCELoss() # 도메인 분류용
        
    # result : DANN_CNN에서 반환된 값
    # label : 숫자 0 ~ 9에 대한 라벨
    # domain_num : 0(source) or 1(target)
    def forward(self, result, label, domain_num, alpha = 1):
        label_logits, domain_logits = result # DANN_CNN의 결과

        batch_size = domain_logits.shape[0]

        # print("segment_mask : ", label.shape)
        # print("domain_answer : ", domain_target.shape)
        segment_loss = self.CE(label_logits, label) # class 분류 loss

        # domain_target = torch.FloatTensor([domain_num] * batch_size).unsqueeze(1).to(device)
        # domain_loss = self.BCE(domain_logits, domain_target)

        domain_target = torch.LongTensor([domain_num] * batch_size).to(device)
        domain_loss = self.CE(domain_logits, domain_target) # domain 분류 loss
        
        loss = segment_loss + alpha * domain_loss

        return loss, segment_loss, domain_loss
    

loss_fn = DANN_Loss().to(device)

#criterion =nn.CrossEntropyLoss()
#domain_criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss(weight=class_weights)

In [12]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [13]:
LR = 0.001
EP = 25
BATCH_SIZE = 16
ACCMULATION_STEP = 1 
N_CLASSES = 13 #IoU 점수측정하기 위한 클래스의 개수
ALPHA = 0.5
# model 초기화
model = Resnet18_Unet(n_classes = N_CLASSES).to(device)
#model = Resnet50_Unet(n_classes = N_CLASSES).to(device)
#model = Unet(n_classes = N_CLASSES).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

optimizer.zero_grad() 

source_dataset = CustomDataset(csv_file='./data/896_csv/train_source.csv', transform=transform)
source_dataloader = DataLoader(source_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_target_dataset = CustomDataset(csv_file='./data/896_csv/val_source.csv', transform=transform)
val_target_dataloader = DataLoader(val_target_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [14]:
# import pandas as pd
# import os
# import cv2
# import matplotlib.pyplot as plt
# a1 = pd.read_csv("./data/6_fish_source.csv")
# p = "./data/224/"
# a2 = os.path.join(p, a1.iloc[3,2])
# a3 = cv2.imread(a2)
# a4 = cv2.cvtColor(a3, cv2.COLOR_BGR2GRAY)
# a4 = np.round(a4).astype(np.uint8)
# a5 = a4*20

# plt.imshow(a5, cmap='gray')
# plt.axis('off')
# plt.show()

In [15]:
# import random
# #torch.cuda.empty_cache()
# # import wandb


# # wandb.init(
# #     # set the wandb project where this run will be logged
# #     project="practice_10_27_4d_res18",
    
# #     # track hyperparameters and run metadata
# #     config={
# #     "learning_rate": LR,
# #     "architecture": "CNN",
# #     "dataset": "Samsung",
# #     "epochs": EP,
# #     }
# # )

# for epoch in range(EP):
#     # 클래스별 IoU를 누적할 리스트 초기화
#     train_class_ious = []
#     fish_train_class_ious = []
#     # 학습
#     model.train()
#     epoch_loss = 0
#     seg_loss = 0
#     dom_loss = 0

#     for source_images, source_masks in tqdm(source_dataloader):
#         label = random.randint(0,3)
#         source_images, source_masks = apply_fisheye_distortion(source_images, source_masks, label)
#         source_images = source_images.float().to(device)
#         source_masks = source_masks.long().to(device)

#         optimizer.zero_grad()
#         source_outputs = model(source_images)

#         source_loss, segment_loss, domain_loss = loss_fn(source_outputs, source_masks, label, alpha = ALPHA)

#         loss = source_loss
#         epoch_loss += loss.item()
#         seg_loss += segment_loss.item()
#         dom_loss += domain_loss.item()

#         loss.backward()
#         optimizer.step()
#         #scheduler.step()
#                 # train 클래스별 IoU 계산
#         source_outputs = torch.softmax(source_outputs[0], dim=1).cpu()
#         source_outputs = torch.argmax(source_outputs, dim=1).numpy()

#         for class_id in range(N_CLASSES):
#             iou = calculate_iou_per_class(np.array(source_masks.cpu()), np.array(source_outputs), class_id)
#             train_class_ious.append(iou)

#     train_class_ious = np.array(train_class_ious).reshape(N_CLASSES, -1)
#     train_class_ious = np.mean(train_class_ious, axis=1)
#     print("--IoU Scores Train--")
#     for class_id, iou in enumerate(train_class_ious):
#         print(f'Class{class_id}: {iou:.4f}', end=" ")
#         if (class_id+1) % 7 == 0:
#             print()

#     # mIoU 계산
#     train_mIoU = np.mean(train_class_ious)

#     # 에폭마다 결과 출력 
#     print(f"\nEpoch{epoch+1}")
#     print(f"Train seg Loss: {(seg_loss/len(source_dataloader))}")
#     print(f"Train dom Loss: {(dom_loss/len(source_dataloader))}")
#     print(f"Train Loss: {(epoch_loss/len(source_dataloader))}")
#     print(f"Train mIoU: {train_mIoU}" )
#     print("___________________________________________________________________________________________\n")


#     ################################################################
#     # 클래스별 IoU를 누적할 리스트 초기화
#     val_class_ious = []
#     fish_val_class_ious = []
#     val_epoch_loss = 0
#     val_seg_loss = 0
#     val_dom_loss = 0
#     # 학습
#     with torch.no_grad():
#         model.eval()

#         for target_images, target_masks in tqdm(val_target_dataloader):
#             label = 2.5
#             target_images, target_masks = apply_fisheye_distortion(target_images, target_masks, label)
#             target_images = target_images.float().to(device)
#             target_masks = target_masks.long().to(device)

#             target_outputs = model(target_images)

#             target_loss, val_segment_loss, val_domain_loss = loss_fn(target_outputs, target_masks, 0, alpha = ALPHA)

#             loss = target_loss

#             val_epoch_loss += loss.item()
#             val_seg_loss += val_segment_loss.item()
#             val_dom_loss += val_domain_loss.item()

#             # train 클래스별 IoU 계산
#             target_outputs = torch.softmax(target_outputs[0], dim=1).cpu()
#             target_outputs = torch.argmax(target_outputs, dim=1).numpy()

#             for class_id in range(N_CLASSES):
#                 iou = calculate_iou_per_class(np.array(target_masks.cpu()), np.array(target_outputs), class_id)
#                 fish_val_class_ious.append(iou)

#     fish_val_class_ious = np.array(fish_val_class_ious).reshape(N_CLASSES, -1)
#     fish_val_class_ious = np.mean(fish_val_class_ious, axis=1)
#     print()
#     print("--IoU Scores Fish val--")
#     for class_id, iou in enumerate(fish_val_class_ious):
#         print(f'Class{class_id}: {iou:.4f}', end=" ")
#         if (class_id+1) % 7 == 0:
#             print()

#     # mIoU 계산
#     fish_val_mIoU = np.mean(fish_val_class_ious)

#     # 에폭마다 결과 출력 
#     print(f"\nEpoch{epoch+1}")
#     print(f"Valid seg Loss: {(val_seg_loss/len(val_target_dataloader))}")
#     print(f"Valid dom Loss: {(val_dom_loss/len(val_target_dataloader))}")
#     print(f"Valid Loss: {(val_epoch_loss/len(val_target_dataloader))}")
#     print(f"Valid mIoU: {fish_val_mIoU}" )
#     print("___________________________________________________________________________________________\n")

# #     # log metrics to wandb
# #     wandb.log({"train score": train_mIoU})
# #     wandb.log({"val score": fish_val_mIoU})
# #     wandb.log({"train loss": (epoch_loss/len(source_dataloader))})
# #     wandb.log({"val loss": (val_epoch_loss/len(val_target_dataloader))})
    
    
# # # [optional] finish the wandb run, necessary in notebooks
# # wandb.finish()


In [16]:
import random
#torch.cuda.empty_cache()
# import wandb


# wandb.init(
#     # set the wandb project where this run will be logged
#     project="practice_10_27_4d_res18",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": LR,
#     "architecture": "CNN",
#     "dataset": "Samsung",
#     "epochs": EP,
#     }
# )

for epoch in range(EP):
    # 클래스별 IoU를 누적할 리스트 초기화
    train_class_ious = []
    fish_train_class_ious = []
    # 학습
    model.train()
    epoch_loss = 0
    seg_loss = 0
    dom_loss = 0

    for source_images, source_masks in tqdm(source_dataloader):
        for label in range(4):
            source_image, source_mask = apply_fisheye_distortion(source_images, source_masks, label)
            source_image = source_image.float().to(device)
            source_mask = source_mask.long().to(device)

            optimizer.zero_grad()
            source_outputs = model(source_image)

            source_loss, segment_loss, domain_loss = loss_fn(source_outputs, source_mask, label, alpha = ALPHA)

            loss = source_loss
            epoch_loss += loss.item()
            seg_loss += segment_loss.item()
            dom_loss += domain_loss.item()
                                
        loss.backward()
        optimizer.step()
        #scheduler.step()
                # train 클래스별 IoU 계산
        source_outputs = torch.softmax(source_outputs[0], dim=1).cpu()
        source_outputs = torch.argmax(source_outputs, dim=1).numpy()

        for class_id in range(N_CLASSES):
            iou = calculate_iou_per_class(np.array(source_masks.cpu()), np.array(source_outputs), class_id)
            train_class_ious.append(iou)

    train_class_ious = np.array(train_class_ious).reshape(N_CLASSES, -1)
    train_class_ious = np.mean(train_class_ious, axis=1)
    print("--IoU Scores Train--")
    for class_id, iou in enumerate(train_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print()

    # mIoU 계산
    train_mIoU = np.mean(train_class_ious)

    # 에폭마다 결과 출력 
    print(f"\nEpoch{epoch+1}")
    print(f"Train seg Loss: {(seg_loss/len(source_dataloader))}")
    print(f"Train dom Loss: {(dom_loss/len(source_dataloader))}")
    print(f"Train Loss: {(epoch_loss/len(source_dataloader))}")
    print(f"Train mIoU: {train_mIoU}" )
    print("___________________________________________________________________________________________\n")


    ################################################################
    # 클래스별 IoU를 누적할 리스트 초기화
    val_class_ious = []
    fish_val_class_ious = []
    val_epoch_loss = 0
    val_seg_loss = 0
    val_dom_loss = 0
    # 학습
    with torch.no_grad():
        model.eval()

        for target_images, target_masks in tqdm(val_target_dataloader):
            label = 2.5
            target_images, target_masks = apply_fisheye_distortion(target_images, target_masks, label)
            target_images = target_images.float().to(device)
            target_masks = target_masks.long().to(device)

            target_outputs = model(target_images)

            target_loss, val_segment_loss, val_domain_loss = loss_fn(target_outputs, target_masks, 0, alpha = ALPHA)

            loss = target_loss

            val_epoch_loss += loss.item()
            val_seg_loss += val_segment_loss.item()
            val_dom_loss += val_domain_loss.item()

            # train 클래스별 IoU 계산
            target_outputs = torch.softmax(target_outputs[0], dim=1).cpu()
            target_outputs = torch.argmax(target_outputs, dim=1).numpy()

            for class_id in range(N_CLASSES):
                iou = calculate_iou_per_class(np.array(target_masks.cpu()), np.array(target_outputs), class_id)
                fish_val_class_ious.append(iou)

    fish_val_class_ious = np.array(fish_val_class_ious).reshape(N_CLASSES, -1)
    fish_val_class_ious = np.mean(fish_val_class_ious, axis=1)
    print()
    print("--IoU Scores Fish val--")
    for class_id, iou in enumerate(fish_val_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print()

    # mIoU 계산
    fish_val_mIoU = np.mean(fish_val_class_ious)

    # 에폭마다 결과 출력 
    print(f"\nEpoch{epoch+1}")
    print(f"Valid seg Loss: {(val_seg_loss/len(val_target_dataloader))}")
    print(f"Valid dom Loss: {(val_dom_loss/len(val_target_dataloader))}")
    print(f"Valid Loss: {(val_epoch_loss/len(val_target_dataloader))}")
    print(f"Valid mIoU: {fish_val_mIoU}" )
    print("___________________________________________________________________________________________\n")

#     # log metrics to wandb
#     wandb.log({"train score": train_mIoU})
#     wandb.log({"val score": fish_val_mIoU})
#     wandb.log({"train loss": (epoch_loss/len(source_dataloader))})
#     wandb.log({"val loss": (val_epoch_loss/len(val_target_dataloader))})
    
    
# # [optional] finish the wandb run, necessary in notebooks
# wandb.finish()


100%|██████████| 138/138 [07:30<00:00,  3.27s/it]


--IoU Scores Train--
Class0: 0.0250 Class1: 0.0421 Class2: 0.0619 Class3: 0.0771 Class4: 0.0881 Class5: 0.0957 Class6: 0.0989 
Class7: 0.1015 Class8: 0.1024 Class9: 0.1042 Class10: 0.1042 Class11: 0.1092 Class12: 0.1068 
Epoch1
Train seg Loss: 8.043044443363728
Train dom Loss: 4120.223648688067
Train Loss: 2068.154870762989
Train mIoU: 0.0859469899195324
___________________________________________________________________________________________



100%|██████████| 30/30 [00:33<00:00,  1.10s/it]



--IoU Scores Fish val--
Class0: 0.3272 Class1: 0.2622 Class2: 0.3152 Class3: 0.3432 Class4: 0.2880 Class5: 0.3192 Class6: 0.3506 
Class7: 0.2695 Class8: 0.3118 Class9: 0.3158 Class10: 0.2984 Class11: 0.2929 Class12: 0.3071 
Epoch1
Valid seg Loss: 0.34410766859849296
Valid dom Loss: 1621.4189575195312
Valid Loss: 811.0535827636719
Valid mIoU: 0.3077791571180629
___________________________________________________________________________________________



100%|██████████| 138/138 [07:48<00:00,  3.39s/it]


...


--IoU Scores Fish val--
Class0: 0.4137 Class1: 0.3375 Class2: 0.3893 Class3: 0.4153 Class4: 0.3419 Class5: 0.3843 Class6: 0.4222 
Class7: 0.3500 Class8: 0.3789 Class9: 0.3763 Class10: 0.3734 Class11: 0.3694 Class12: 0.3897 
Epoch24
Valid seg Loss: 0.3617448717355728
Valid dom Loss: 1330.7086588541667
Valid Loss: 665.716074625651
Valid mIoU: 0.3801467394268263
___________________________________________________________________________________________



100%|██████████| 138/138 [02:14<00:00,  1.03it/s]


--IoU Scores Train--
Class0: 0.1566 Class1: 0.1660 Class2: 0.1684 Class3: 0.1635 Class4: 0.1678 Class5: 0.1577 Class6: 0.1682 
Class7: 0.1619 Class8: 0.1649 Class9: 0.1670 Class10: 0.1715 Class11: 0.1666 Class12: 0.1689 
Epoch25
Train seg Loss: 7.605005503078734
Train dom Loss: 3241.8818520808563
Train Loss: 1628.545929117695
Train mIoU: 0.1653107699461978
___________________________________________________________________________________________



100%|██████████| 30/30 [00:08<00:00,  3.44it/s]


--IoU Scores Fish val--
Class0: 0.4272 Class1: 0.3558 Class2: 0.4019 Class3: 0.4294 Class4: 0.3593 Class5: 0.3964 Class6: 0.4374 
Class7: 0.3602 Class8: 0.3916 Class9: 0.3937 Class10: 0.3854 Class11: 0.3782 Class12: 0.4020 
Epoch25
Valid seg Loss: 0.34594605565071107
Valid dom Loss: 1389.3079956054687
Valid Loss: 694.9999430338542
Valid mIoU: 0.3937357508919259
___________________________________________________________________________________________



In [18]:
torch.save(model.state_dict(), './data/resnet_unet_soft4_25ep.pth')